In [1]:
import pickle
import os
import pandas as pd
import enum
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from spear.labeling import PreLabels
import numpy as np
from spear.cage import Cage
from utils import custom_dataset, train_all_LF, get_various_data
from generate_LF import get_variables
from spear.labeling import labeling_function, ABSTAIN, preprocessor, continuous_scorer
import re


2023-05-28 10:03:25.017744: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 10:03:25.091262: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-28 10:03:25.464433: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-28 10:03:25.464473: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [2]:
from labelling_function import *
from PIL import Image
from resnet18 import ResNet, BasicBlock
import torch
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
from resnet18 import ResNet, BasicBlock
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import cv2

In [3]:
ABSTAIN = None

class ClassLabels(enum.Enum):
    LYMPHOCYTE= 0
    NONLYMPHOCYTE = 1
    
    
   

In [4]:
from spear.labeling import labeling_function, ABSTAIN, preprocessor, continuous_scorer
import re
# path = "/home/akshit/Desktop/MICCAI/data/models/"
path = "/home/raja/Desktop/MICCAI/data/models/100/"

# SVM
@continuous_scorer()
def svm_V1(x,**kwargs):
    import pickle
    import numpy as np

    #x = np.array(x).flatten() # x is 28x28x3 input img 
    svm = pickle.load(open(path+'0_svmV11.pkl','rb'))
    confidence_scores = svm.predict_proba([x])
    # print(confidence_scores)
    return float(confidence_scores[0][0]) #only called when model doesn't ABSTAIN, output confidence on class label

@labeling_function(cont_scorer=svm_V1, label=ClassLabels.LYMPHOCYTE)
def LF_svm0_V1(x, **kwargs):
    import pickle
    import numpy as np

    #x = np.array(x).flatten()  
    svm = pickle.load(open(path+'0_svmV11.pkl','rb'))
    
    if svm.predict_proba([x])[0][0]>0.50: 
        return ClassLabels.LYMPHOCYTE # Return label only if confidence > 0.8
    else: 
        return ABSTAIN
    



In [5]:
# SVM
@continuous_scorer()
def svm1_V1(x,**kwargs):
    import pickle
    import numpy as np

    #x = np.array(x).flatten() # x is 28x28x3 input img 
    svm = pickle.load(open(path+'1_svmV11.pkl','rb'))
    confidence_scores = svm.predict_proba([x])

    return float(confidence_scores[0][1]) #only called when model doesn't ABSTAIN, output confidence on class label

@labeling_function(cont_scorer=svm1_V1, label=ClassLabels.NONLYMPHOCYTE)
def LF_svm1_V1(x, **kwargs):
    import pickle
    import numpy as np

    #x = np.array(x).flatten()  
    svm = pickle.load(open(path+'1_svmV11.pkl','rb'))
    
    if svm.predict_proba([x])[0][1]>0.6: 
        return ClassLabels.NONLYMPHOCYTE
    else: 
        return ABSTAIN


In [6]:
from labelling_function import *

@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL1(c,**kwargs): 
    red_score = calculate_brown_score(c)
    blue_score = calculate_blue_score(c)
    green_score = calculate_green_score(c)
    if blue_score > 155:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN
    
@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL2(c,**kwargs): 
    red_score = calculate_brown_score(c)
    blue_score = calculate_blue_score(c)
    if 10 < blue_score < 180 and red_score > 170:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN    
    
@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL9(c,**kwargs): 
    green_score = calculate_green_score(c)
    blue_score = calculate_blue_score(c)
    if 10 < blue_score < 180 and green_score > 155:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN      
    
@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL3(c,**kwargs): 
    brown_score = compute_brown_score(c)
    total_blue_score = compute_blue_score(c)
    if total_blue_score < 12:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN   


@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL4(c,**kwargs): 
    brown_score = compute_brown_score(c)
    total_blue_score = compute_blue_score(c)
    if brown_score > 585:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN   

@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL5(c,**kwargs): 
    red_score = calculate_brown_score(c)
    blue_score = calculate_blue_score(c)
    green_score = calculate_green_score(c)
    if red_score > 170:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN


@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL6(c,**kwargs): 
    red_score = calculate_brown_score(c)
    blue_score = calculate_blue_score(c)
    green_score = calculate_green_score(c)
    if red_score < 12:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN

@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL7(c,**kwargs): 
    red_score = calculate_brown_score(c)
    blue_score = calculate_blue_score(c)
    green_score = calculate_green_score(c)
    if green_score > 155:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN

@labeling_function(label=ClassLabels.NONLYMPHOCYTE)
def LFNL8(c,**kwargs): 
    black_score = compute_black_score(c)
    if black_score > 9:
        return ClassLabels.NONLYMPHOCYTE
    else:
        return ABSTAIN           

    
                 

In [7]:
from spear.labeling import LFSet

LFS = [
    #ResNet18LF_LYMPHOCYTE,
    #ResNet18LF_NONLYMPHOCYTE,
    LFNL1,
    LFNL2,
    LFNL3,
    LFNL4,
    LFNL5,
    LFNL6,
    LFNL7,
    LFNL8,
    LFNL9,
    
    LF_svm0_V1,
    LF_svm1_V1,  
  
]

QT2 = 0.99
QC2 = 0.99

qt1 = np.array([0.9999,0.9999,0.9999,0.9999,0.9999,0.9999,0.9999,0.9999,0.9999])
qc1 = np.array([0.99999,0.99999,0.9999,0.9999,0.9999,0.9999,0.9999,0.9999,0.9999])

rules = LFSet("BM_LF")
rules.add_lf_list(LFS)

In [41]:
from utils import custom_dataset, train_all_LF, get_various_data
from generate_LF import get_variables
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

classes,label_frac,data_path,save_path = get_variables()
dataset,X,Y = custom_dataset(classes=classes,path=data_path ,fraction=label_frac)

In [42]:
X.shape

(210, 2700)

In [43]:
len(Y)

210

In [44]:




X, X_feats, Y = X, X, Y

#X = X.reshape((210,30, 30, 3))

validation_size = 2
test_size = 2
L_size = 202
U_size = 4
n_lfs = len(rules.get_lfs())

X_V, Y_V, X_feats_V,_, X_T, Y_T, X_feats_T,_, X_L, Y_L, X_feats_L,_, X_U, X_feats_U,_ = get_various_data(X, Y, X_feats, n_lfs, validation_size, test_size, L_size, U_size)



In [45]:
path_json = './jl_pkl/sms_json.json'
V_path_pkl = './jl_pkl/sms_pickle_V.pkl' #validation data - have true labels
T_path_pkl = './jl_pkl/sms_pickle_T.pkl' #test data - have true labels
L_path_pkl = './jl_pkl/sms_pickle_L.pkl' #Labeled data - have true labels
U_path_pkl = './jl_pkl/sms_pickle_U.pkl' #unlabelled data - don't have true labels

log_path_jl_1 = './jl_pkl/sms_log_1.txt' #jl is an algorithm, can be found below
params_path = './jl_pkl/sms_params.pkl' #file path to store parameters of JL, used below

In [46]:
from spear.labeling import PreLabels
classes,label_frac,data_path,save_path = get_variables()
train_all_LF(X,Y,len(classes),save_path,label_frac)

sms_noisy_labels = PreLabels(name="lysto",
                               data=X_V,
                               gold_labels=Y_V,
                               data_feats=X_feats_V,
                               rules=rules,
                               labels_enum=ClassLabels,
                               num_classes=2)
sms_noisy_labels.generate_pickle(V_path_pkl)
sms_noisy_labels.generate_json(path_json) #generating json files once is enough

sms_noisy_labels = PreLabels(name="lysto",
                               data=X_T,
                               gold_labels=Y_T,
                               data_feats=X_feats_T,
                               rules=rules,
                               labels_enum=ClassLabels,
                               num_classes=2)
sms_noisy_labels.generate_pickle(T_path_pkl)

sms_noisy_labels = PreLabels(name="lysto",
                               data=X_L,
                               gold_labels=Y_L,
                               data_feats=X_feats_L,
                               rules=rules,
                               labels_enum=ClassLabels,
                               num_classes=2)
sms_noisy_labels.generate_pickle(L_path_pkl)

sms_noisy_labels = PreLabels(name="lysto",
                               data=X_U,
                               rules=rules,
                               data_feats=X_feats_U,
                               labels_enum=ClassLabels,
                               num_classes=2) #note that we don't pass gold_labels here, for the unlabelled data
sms_noisy_labels.generate_pickle(U_path_pkl)

/home/raja/Desktop/cage/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Trained & Saved 6 models


/home/raja/Desktop/cage/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Trained & Saved 6 models


100%|██████████| 4/4 [00:00<00:00, 604.06it/s]


In [47]:
import pickle

# Load the labeled data from the pickle file
with open(U_path_pkl, 'rb') as f:
    labeled_data = pickle.load(f)

# Get the feature matrix from the labeled data
U_L = labeled_data

# Determine the number of features
n_features = U_L.shape

print("Number of features:", n_features)


Number of features: (4, 2700)


In [48]:
from spear.utils import get_data, get_classes

data_U = get_data(path = U_path_pkl, check_shapes=True)
#check_shapes being True(above), asserts for relative shapes of arrays in pickle file
print("Number of elements in data list: ", len(data_U))
print("Shape of feature matrix: ", data_U[0].shape)
print("Shape of labels matrix: ", data_U[1].shape)
print("Shape of continuous scores matrix : ", data_U[6].shape)
print("Total number of classes: ", data_U[9])

classes = get_classes(path = path_json)
print("Classes dictionary in json file(modified to have integer keys): ", classes)


Number of elements in data list:  10
Shape of feature matrix:  (4, 2700)
Shape of labels matrix:  (4, 11)
Shape of continuous scores matrix :  (4, 11)
Total number of classes:  2
Classes dictionary in json file(modified to have integer keys):  {0: 'LYMPHOCYTE', 1: 'NONLYMPHOCYTE'}


In [49]:
from spear.utils import get_data, get_classes

data_U = get_data(path = L_path_pkl, check_shapes=True)
#check_shapes being True(above), asserts for relative shapes of arrays in pickle file
print("Number of elements in data list: ", len(data_U))
print("Shape of feature matrix: ", data_U[0].shape)
print("Shape of labels matrix: ", data_U[1].shape)
print("Shape of continuous scores matrix : ", data_U[6].shape)
print("Total number of classes: ", data_U[9])

classes = get_classes(path = path_json)
print("Classes dictionary in json file(modified to have integer keys): ", classes)


Number of elements in data list:  10
Shape of feature matrix:  (202, 2700)
Shape of labels matrix:  (202, 11)
Shape of continuous scores matrix :  (202, 11)
Total number of classes:  2
Classes dictionary in json file(modified to have integer keys):  {0: 'LYMPHOCYTE', 1: 'NONLYMPHOCYTE'}


In [50]:


from spear.jl import JL

n_features = 2700
n_hidden = 512
feature_model = 'nn'
'''
'nn' is neural network. other alternative is 'lr'(logistic regression) which doesn't need n_hidden to be passed
during initialisation.
''' 

jl = JL(path_json = path_json, n_lfs = n_lfs, n_features = n_features, feature_model = feature_model, \
        n_hidden = n_hidden)



In [57]:
loss_func_mask = [1,1,0,1,0,0,1] 
#loss_func_mask = [0,0,0,0,0,0,1]
'''
One can keep 0s in places where he don't want the specific loss function to be part
the final loss function used in training. Refer documentation(spear.JL.core.JL) to understand
the which index of loss_func_mask refers to what loss function.

Note: the loss_func_mask above may not be the optimal mask for sms dataset. We have to try
      some other masks too, to find the best one that gives good accuracies.
'''
batch_size = 50
lr_fm = 0.0005
lr_gm = 0.01
use_accuracy_score = False

jl = JL(path_json = path_json, n_lfs = n_lfs, n_features = n_features, feature_model = feature_model, n_hidden = n_hidden)

probs_fm, probs_gm = jl.fit_and_predict_proba(path_L = L_path_pkl, path_U = U_path_pkl, path_V = V_path_pkl, path_T = T_path_pkl, loss_func_mask = loss_func_mask, batch_size = batch_size, lr_fm = lr_fm, lr_gm = lr_gm, use_accuracy_score = use_accuracy_score, path_log = log_path_jl_1, return_gm = True, n_epochs = 100, start_len = 7,stop_len = 10, is_qt = True, is_qc = True, qt = 0.9, qc = 0.85, metric_avg = 'binary')

labels = np.argmax(probs_fm, 1)
print("probs_fm shape: ", probs_fm.shape)
print("probs_gm shape: ", probs_gm.shape)

  0%|          | 0/100 [00:00<?, ?it/s]


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [56]:
feature_model = 'nn' #resetting feature_model as 'nn'(neural network) for further trainings

jl = JL(path_json = path_json, n_lfs = n_lfs, n_features = n_features, feature_model = feature_model, \
        n_hidden = n_hidden)

labels_fm, labels_gm = jl.fit_and_predict(path_L = L_path_pkl, path_U = U_path_pkl, path_V = V_path_pkl, \
        path_T = T_path_pkl, loss_func_mask = loss_func_mask, batch_size = batch_size, lr_fm = lr_fm, lr_gm = \
    lr_gm, use_accuracy_score = use_accuracy_score, path_log = log_path_jl_1, return_gm = True, n_epochs = \
    100, start_len = 7,stop_len = 10, is_qt = True, is_qc = True, qt = 0.9, qc = 0.85, metric_avg = 'binary', \
    need_strings = False)

print("labels_fm shape: ", labels_fm.shape)
print("labels_gm shape: ", labels_gm.shape)
print(type(labels_fm[0]))
print(type(labels_gm[0]))


  6%|▌         | 6/100 [00:00<00:10,  9.36it/s]/home/raja/Desktop/cage/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 19%|█▉        | 19/100 [00:02<00:08,  9.00it/s]

early stopping at epoch: 19	best_epoch: 8
score used: f1_score
best_gm_val_score:1.0	best_fm_val_score:0.6666666666666666
best_gm_test_score:1.0	best_fm_test_score:1.0
best_gm_test_precision:1.0	best_fm_test_precision:1.0
best_gm_test_recall:1.0	best_fm_test_recall:1.0
labels_fm shape:  (4,)
labels_gm shape:  (4,)
<class 'numpy.int64'>
<class 'numpy.int64'>


In [54]:
jl = JL(path_json = path_json, n_lfs = n_lfs, n_features = n_features, feature_model = feature_model, \
        n_hidden = n_hidden)

labels_fm, labels_gm = jl.fit_and_predict(path_L = L_path_pkl, path_U = U_path_pkl, path_V = V_path_pkl, \
        path_T = T_path_pkl, loss_func_mask = loss_func_mask, batch_size = batch_size, lr_fm = lr_fm, lr_gm = \
    lr_gm, use_accuracy_score = use_accuracy_score, path_log = log_path_jl_1, return_gm = True, n_epochs = \
    100, start_len = 7,stop_len = 10, is_qt = True, is_qc = True, qt = 0.9, qc = 0.85, metric_avg = 'binary', \
    need_strings = True)

print("labels_fm shape: ", labels_fm.shape)
print("labels_gm shape: ", labels_gm.shape)
print(type(labels_fm[0]))
print(type(labels_gm[0]))


  0%|          | 0/100 [00:00<?, ?it/s]/home/raja/Desktop/cage/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  2%|▏         | 2/100 [00:00<00:09,  9.89it/s]/home/raja/Desktop/cage/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  4%|▍         | 4/100 [00:00<00:09,  9.83it/s]/home/raja/Desktop/cage/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

early stopping at epoch: 19	best_epoch: 8
score used: f1_score
best_gm_val_score:1.0	best_fm_val_score:0.0
best_gm_test_score:1.0	best_fm_test_score:1.0
best_gm_test_precision:1.0	best_fm_test_precision:1.0
best_gm_test_recall:1.0	best_fm_test_recall:1.0
labels_fm shape:  (4,)
labels_gm shape:  (4,)
<class 'numpy.str_'>
<class 'numpy.str_'>
